## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [23]:
import glob

import numpy as np
import scipy as sp
from scipy import signal
import scipy.io
from matplotlib import pyplot as plt
from matplotlib import mlab

fs=125 #sampling rate
minfreq=0.66 #40 bpm
maxfreq=4 #240


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

    
def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
        
    # Compute pulse rate estimates and estimation confidence.
    errors, confidence = estimatePulseRate(ref_fl,ppg, accx, accy, accz)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidence

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)



##### My implementation ,the code in above section is skeletal code provided by udacity.

Knowledge links used for reference: 

https://knowledge.udacity.com/questions/314330

https://knowledge.udacity.com/questions/399609

https://knowledge.udacity.com/questions/246546



In [24]:
def get_spectogram(sig):
    """
    Gets spectrogram and frequencies from given signal. 
    Calculate the magnitude of the acceleration.
    Arguments: 
        sig: required sensor signal (e.g. ppg or acc)
    Returns:
        spec: ndarray
        freqs: ndarray
    adapted from https://knowledge.udacity.com/questions/399609
    refered for understanding https://knowledge.udacity.com/questions/246546
    refered to https://vibrationresearch.com/blog/what-is-a-spectrogram/
    """
    spec, freqs, t=mlab.specgram(sig,NFFT=fs*8,Fs=fs,noverlap=6*fs, pad_to=10*fs)
    spec=spec[(freqs >= minfreq) & (freqs <= maxfreq)]
    freqs=freqs[(freqs >= minfreq) & (freqs <= maxfreq)]
    
    return spec, freqs

def BandpassFilter(sig): 
    """
    BandpassFilter function  allows certain frequency range only.
    Refer to ReadMe.md file for passband range.
    the Readme file assumes pulse rate will be restricted between 40BPM (beats per minute) and 240BPM
    Banpass Filter code has been adapted from Lowpassfilter in Lesson 3 
    Returns:
        Filtered signal
    """
    pass_band=(minfreq,maxfreq) 
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, sig)

def calcSNR(sig, est_freq):
    """
    Calculate Signal to noise ratio to determine how clean the signal is 
    Refered to Lesson 3 Exercise 3 Solution
    SNR was recommended to be used in https://knowledge.udacity.com/questions/399609
    """
    n = len(sig)*2
    #frequency of first harmonic
    harmonic_f = est_freq * 2
    
    # do Fourier Transform
    fft_mag = np.abs(np.fft.rfft(sig, n))
    freqs = np.fft.rfftfreq(n, 1/fs)
    
    #Find the frequencies close to heart rate and first harmonic of the heart rate
    window_f = 5/60
    fundamental_frequency_window = (freqs > est_freq - window_f) & (freqs < est_freq + window_f)
    harmonic_frequency_window = (freqs > harmonic_f - window_f) & (freqs < harmonic_f + window_f)
    
    #Calculate signal and noise power
    sig = np.sum(fft_mag[(fundamental_frequency_window) | (harmonic_frequency_window)])
    noise = np.sum(fft_mag[~ ((fundamental_frequency_window) | (harmonic_frequency_window))])
    snr = sig / noise
    
    return snr

def Featurize(ppg,accx, accy, accz,fs=fs):
    """Featurization of the accelerometer and ppg signal.
        Adapted from Lesson 4 .
      Args:
      accx: (np.array) x-channel of the accelerometer.
      accy: (np.array) y-channel of the accelerometer.
      accz: (np.array) z-channel of the accelerometer.
      fs: (number) the sampling rate of the accelerometer
      ppg:  (np.array) ppg signal

      Returns:
       n-tuple of accelerometer and ppg features
    """
    # Use Bandpass Filter to take out signals below 40 bpm and above 240 bpm
    ppg=BandpassFilter(ppg)
    accx=BandpassFilter(accx)
    accy=BandpassFilter(accy)
    accz=BandpassFilter(accz)
    
    #Get spectrogram frequencies
    spec_ppg, freqs_ppg = get_spectogram(ppg)
    spec_accx, freqs_accx = get_spectogram(accx)
    spec_accy, freqs_accy = get_spectogram(accy)
    spec_accz, freqs_accz = get_spectogram(accz)
      
    #Get frequency and time
    freqs = freqs_ppg.shape[0]
    t_steps = spec_ppg.shape[1]
    
    # Get indices for largest signal peaks
    ppg_ind = (-spec_ppg).argsort(axis=0)
    accx_ind = (-spec_accx).argsort(axis=0)
    accy_ind = (-spec_accy).argsort(axis=0)
    accz_ind = (-spec_accz).argsort(axis=0)
    
    return (freqs,t_steps,spec_ppg, freqs_ppg,
            ppg_ind,accx_ind,accy_ind,accz_ind)
           
def estimatePulseRate(ref_f1,ppg,accx, accy, accz,fs=fs): #refer to Lesson 4
    """Estimates the Pulse Rate of the accelerometer and ppg signal.
        Adapted from Lesson 4 .
        refered for understanding https://knowledge.udacity.com/questions/246546
      Args:
      accx: (np.array) x-channel of the accelerometer.
      accy: (np.array) y-channel of the accelerometer.
      accz: (np.array) z-channel of the accelerometer.
      fs: (number) the sampling rate of the accelerometer
      ppg:  (np.array) ppg signal

    Returns:
        errors, confidence numpy array
    """
    # get Ground Truth 
    # Load Reference
    groundtruth = scipy.io.loadmat(ref_f1)["BPM0"].reshape(-1)
    
    #Extract strongest frequencies in ppg and accelerometer signals 
    (freqs,t_steps,spec_ppg, freqs_ppg,
     ppg_ind,accx_ind,accy_ind,accz_ind) = Featurize(ppg,accx, accy, accz)
    
    # Estimate Pulse Rates adapted from https://knowledge.udacity.com/questions/399609
    #
    estimated_freq=[]
    for t in range(t_steps):
        for freq in range(freqs):
            i=0
            if freq == 2:# if just peaks at 2 frequencies just pick ppg
                estimated_freq.append(freqs_ppg[ppg_ind[freq][t]])
                break
            # try to remove accelerometer peaks to identify only ppg
            # we are just doing our best to remove peaks caused due to ARM movements
            elif np.all([(freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i][t]]), 
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i][t]]), 
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i+1][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accx_ind[i+2][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accy_ind[i+2][t]]),
                      (freqs_ppg[ppg_ind[freq][t]] != freqs_ppg[accz_ind[i+2][t]])]):
                estimated_freq.append(freqs_ppg[ppg_ind[freq][t]])
                break
    # use SNR from Lesson 3 to determine "purity" of the ppg signal this is what our confidence is
    confidence = []
    for i in range(len(estimated_freq)):
        snr = calcSNR(ppg, estimated_freq[i])
       # print("For {}, estimated_freq={} and snr={}".format(i, estimated_freq[i],snr))
        confidence.append(snr)
    
    predicted = np.array(estimated_freq) * 60
    
    #Get Error and Confidence in array
    error_arr = np.abs(groundtruth - predicted)
    conf_arr = np.array(confidence)
    #for i in range(len(conf_arr)):
    #    print("At index {}, confidence={} and error={}".format(i, estimated_freq[i],snr))
    return error_arr, conf_arr
    

In [25]:
Evaluate()

19.877438899740184

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

##### Code Description

The first block of code is the starter code provided by UDACITY Team to load and read from the Troika dataset, a test set of data as well as reference(Ground Truth) data. 

Evaluate() is the final Wrapper function which outputs the aggregated error.

Evaluate() calls below functions:
1. LoadTroikaDataset() LoadTroikaDataFile(): to retrieve the data and reference filenames of the data to be processed. It then iterates across the data and reference files to extract PPG and accelerometer signals

2. AggregateErrorMetric() : Returns mean absolute error based on confidense estiamtes

3. RunPulseRateAlgorithm(): This is the crux of the code . IT iterates through ppg and accelerometer signals to return estiamted Pulse Rate.

The second section of the code is where I made the changes. RunPulseRateAlgorithm() calls estimatePulseRate() to collect estimated pulse rate and deviation/error with regards to reference (ground truth).

estimatePulseRate() calls Featurize() function to extract largest peaks at frequencies and over a time range which are basically the features for evaluation of the algorithm. 
estimatePulseRate() then iterates through all the peak frequencies to determine the second best peak by segregating acelerometer peaks.

error is basically deviation between groundtruth/reference frequencies and estimated frequencies.
Confidence is measured by how clean our signal is which can be calculated by determining Signal to Noise Ratio getSNR(). Higher the ratio cleaner the signal is.

Once RunPulseRateAlgorithm() evalautes all the 8 second windows it returns error and confidence arrays to Evaluate() which then  aggregates the results and loads new file. This process is repeated until all files are processed.

##### Data Description
Please refer to Readme.pdf in data set folder for more details.

The algorithm is designed to work on the Troika data set referenced above and noted here as well. The troika dataset includes two data files for each subject, the data file and a reference file. The data file contains 4 arrays of sensor data: one PPG sensor recording and 3 accelermometers (x, y, z axis). The sensors are recording at 8Hz or 125ms time frames. These are extracted in each file. The reference data file contains BPM recordings for 8 second windows recorded every 2 second. Each pair of files represents one test subject starting from rest and starting to run at 15km/hr. This shows BPM starting at resting rate and raising to a high activity rate.

There is no Demographic information, such as Age,Gender,Race details which makes it difficult to be used for any application.


Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

##### Algorithm Description

The algorithm is designed to be execuited within the RunPulseRateAlgorithm() method. Within this method the data is broken down into 8 second windows for processing with the algorithm. The algorithm then uses the featurize() method to determine the important data points for evaluate.

Featurize(): First a low frequency bandpass filter is applied to remove signals below .66Hz and above 4Hz. As we are trying to find the peak associated with heartrate and we know the heartrate should not normally be below 40 BPM and above 240 BPM those frequencies are removed. Next we take the PPG, and all 3 accelerometer signals and create FFT to analyze in the frequency domain. From there, the method identifies the highest frequency signal in PPG, Acc_X, Acc_Y, and Acc_Z to return for evaluation. 

Next the algorithm attempts to identify the right frequency for the heartbeat. If 2 stongest  peaks are identified, the second is chosen for heart rate. However if more peaks are identified, we chose the peak that does not fall under ppg and accelerometer . This step helps eliminate accelerometer noise in the overall signal.

One a frequency is selected, it is converted into a BPM rating and compared to the reference value to determine the error. Lastly for the confidence, we use SNR.

Finally the error and confidences are returned from RunPulseRateAlgorithm()

The algorithm assumes that the second highest peak is a good estimation of Pulse Rate. However this might not always be true. When we have several signals especially accelerometer or EKG it gets harder to differentiate. Also please note that melanin on skin impacts PPG signals, so we might not get a good estiamted heart rate for a subject with darker skin tone.

I believe we might need a more sophesticated process to estimate heart rate from PPG signals given the noise that comes along with this signal.

##### Algorithm Performance
Performance measure is basically determined by deviation between estimated BPM and reference BPM. The Evaluate() returned an aggregated value of 19.87. The lower this value better our Algorithm performs. Some of the ways for improving would be a better logic to isolate the accelorometer peaks from PPG. The algorithm also assumes that hearbeat is basically 2nd peak in ppg which might not always be true especially during resting period or low activity interval. PPG signals are also impacted by melanin in skin tone, higher the melanin inaccurate the PPG peaks become.


Given the caveats in the Data Description section it is hard to consider this a truely generalize algorithm.There is no Demographic information, such as Age,Gender,Race details which makes it difficult to be used for any application.
 It is a useful algorithm during tranisiton from rest to high activity. However cannot be used as a generalized use outside the transition window.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 